In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import jacobi as Jacobi

def show(M,n,a,b): 
    print(M(n,a,b).A)
    print()

$$
\text{P}^{(a,b)}(z) \ = \ \left[ P_{0}^{(a,b)}(z), \ldots,  P_{n}^{(a,b)}(z) , \ldots\right]
$$

<br>

$$
\text{P}^{(a,b)}(-z) \ = \ \text{P}^{(b,a)}(z)\, \Pi
$$

<br>


$$
1 \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a+1,b)}(z)\, A_{a,b}^{+}, \qquad (1-z) \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a-1,b)}(z)\, A_{a,b}^{-}
$$

<br>

$$
1 \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a,b+1)}(z)\, B_{a,b}^{+}, \qquad
(1+z) \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a,b-1)}(z)\, B_{a,b}^{-}
$$


<br>

$$
\left[a + (1+z)\frac{d}{dz} \right] \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a+1,b-1)}(z)\, C_{a,b}^{+}, \quad
\left[b - (1-z)\frac{d}{dz} \right] \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a-1,b+1)}(z)\, C_{a,b}^{-}
$$


<br>

$$
\frac{d}{dz} \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a+1,b+1)}(z)\, D_{a,b}^{+},\quad
\left[ b(1-z) - a(1+z)  +(1-z^{2})\frac{d}{dz}\right] \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a-1,b-1)}(z)\, D_{a,b}^{-}
$$

In [ ]:
n,a,b = 100,0,0

z,w = Jacobi.quadrature(n+1,a,b)

P = Jacobi.polynomials(n+1,a,b,z)

plt.plot(z,P[12])

In [ ]:
A  = Jacobi.operator('A')
B  = Jacobi.operator('B')
C  = Jacobi.operator('C')
D  = Jacobi.operator('D')
Z  = Jacobi.operator('Z')
Pi = Jacobi.operator('Pi')

In [ ]:
nab = n,a,b = (4,4,3/2)

def norm(X,eps=1e-12): 
    print(np.max(np.abs(X(*nab))) < eps)

# I've hijacked the * for the Lie bracket.
# X * Y = X @ Y - Y @ X
M = D(+1) * D(-1)
S = C(+1) * C(-1)

norm(M * S)


for p in (-1,1):
    
    norm( Pi @ C(p) @ Pi - C(-p) )
    norm( Pi @ D(p) @ Pi + D(p) )
    norm( Pi @ A(p) @ Pi - B(p) )
    norm( Pi @ B(p) @ Pi - A(p) )
    
    norm( Z * C(p)  + p * A(p) @ B(-p) )
    norm( Z * D(p)  + p * A(p) @ B(+p) )
    
    norm( M * D(p) - 2*p*D(p) )
    norm( M * C(p)            )
    norm( M * A(p) -   p*A(p) )
    norm( M * B(p) -   p*B(p) )
    
    norm( S * D(p)            )
    norm( S * C(p) - 2*p*C(p) )
    norm( S * A(p) -   p*A(p) )
    norm( S * B(p) +   p*B(p) )
    
    norm( D(p) * C(+p) )
    norm( D(p) * C(-p) )
    
    norm( A(p) * D(+p)           )
    norm( A(p) * D(-p) + p*B(-p) )
    norm( A(p) * C(+p)           )
    norm( A(p) * C(-p) + p*B(+p) )
    
    norm( B(p) * D(+p)           )
    norm( B(p) * D(-p) - p*A(-p) )
    norm( B(p) * C(+p) + p*A(+p) )
    norm( B(p) * C(-p)           )
    
    norm( A(p) * B(+p) )
    norm( A(p) * B(-p) )

In [ ]:
DD = D(+1) @ D(-1)
print(Z.codomain)
print(DD.codomain)

L = DD + Z

print(L.codomain)
print()

show(DD,*nab)
show(Z,*nab)
show(L,*nab)